In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2 as cv
import numpy as np
from sklearn.decomposition import PCA
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
image_path = "/content/drive/MyDrive/A pepo/DS-10/DS-10"

In [ ]:
#Ekstraksi fitur
def ekstraksi_fitur(image_path):
    try:
        image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Image {image_path} tidak bisa dibaca. Skip Image.")
            return None
        image = cv.resize(image, (64, 64))
        features, _ = hog(image,
                          orientations=9,
                          pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2),
                          block_norm='L2-Hys',
                          visualize=True
                          )
        return features
    except Exception:
        print(f"Iamge {image_path} Erorr : {Exception}")
        return None

In [ ]:
#Load Image
def load_feature(folder, use_hog=True):
    features, labels = [], []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg')):
            filepath = os.path.join(folder, filename)
            if use_hog:
                feature = ekstraksi_fitur(filepath)
            else:
                image = cv.imread(filepath, cv.IMREAD_GRAYSCALE)
                if image is not None:
                    image = cv.resize(image, (64, 64))
                    feature = image.flatten()
                else:
                    print(f"Image {image_path} tidak bisa dibaca. Skip Image.")
                    feature = None
            if feature is not None:
                features.append(feature)
                label = filename.split('_')[0]
                labels.append(int(label) if label.isdigit() else 0)
    return np.array(features), np.array(labels)

In [ ]:
def run_experiment(features, labels, split_data, use_feature_extraction):
    print(f"\nSplit data = {split_data} dan Apakah menggunakan feature extraction? : {use_feature_extraction}")

    # Split data
    test_size = 1 - split_data
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=42)

    # Reduksi dimensi dengan PCA untuk data yang menggunakan ekstraksi fitur
    if use_feature_extraction:
        pca = PCA(n_components=50)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

    # Model Training
    models = {
        "SVM": SVC(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Naive Bayes": GaussianNB()
    }

    for model_name, model in models.items():
        print(f"\nTraining model: {model_name}")
        model.fit(X_train, y_train)

        # Output training
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        print("Classification Report:")
        print(classification_report(y_test, y_pred))

        # Tampilkan 10 teratas hasil prediksi model dibandingan data sampel
        print(f"Hasil predict digit dengan test samples menggunakan model = {model_name}:")
        for i in range(min(10, len(y_pred))):
            print(f"Sample {i+1}: True Label = {y_test[i]}, Predicted Label = {y_pred[i]}")

use_hog = True  # load data dengan ekstraksi fitur
features, labels = load_feature(image_path, use_hog=use_hog)

# Normalisasi fitur
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Daftar split ratio dan pilihan penggunaan feature extraction
list_split = [0.6, 0.7, 0.8]
opsi_feature_extractions = [True, False]

# Mulai training
for split_data in list_split:
    for use_feature_extraction in opsi_feature_extractions:
        if not use_feature_extraction:
            # Load ulang data tanpa Feature extraction
            features, labels = load_feature(image_path, use_hog=False)
            features_scaled = scaler.fit_transform(features)
        run_experiment(features_scaled,
                       labels,
                       split_data,
                       use_feature_extraction
                       )

Image /content/drive/MyDrive/A pepo/DS-10/DS-10/6_C_6301102009010_2_6.jpg tidak bisa dibaca. Skip Image.

Split data = 0.6 dan Apakah menggunakan feature extraction? : True

Training model: SVM
Accuracy: 96.11%
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       417
           1       0.98      0.98      0.98       484
           2       0.98      0.97      0.97       414
           3       0.94      0.96      0.95       384
           4       0.98      0.98      0.98       330
           5       0.96      0.95      0.95       307
           6       0.98      0.93      0.96       270
           7       0.97      0.95      0.96       288
           8       0.96      0.93      0.95       270
           9       0.98      0.93      0.95       255

    accuracy                           0.96      3419
   macro avg       0.96      0.96      0.96      3419
weighted avg       0.96      0.96      0.96      3419

Hasil p